Field Interpolation
==============

Given a set of samples of an unkown function, _estimate_ a function $f$


In [ ]:
import seaborn as sns
sns.set_theme()
sns.set(style='darkgrid', context='talk', palette='Pastel1')


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]

$$f(0) = -1$$
$$f(1) = 0.2$$
$$f(2) = 0.9$$
$$f(3) = 2.1$$

Estimate a line, $f(x) = kx + m$, using linear last squares

$$-1 = k*0 + m$$
$$0.2 = k*1 + m$$
$$0.9 = k*2 + m$$
$$2.1 = k*3 + m$$

Or just

$$
\begin{bmatrix}
0 & 1 \\
1 & 1 \\
2 & 1 \\
3 & 1 \\
\end{bmatrix}
\begin{bmatrix} k \\ m \end{bmatrix}
= 
\begin{bmatrix}
-1 \\ 0.2 \\ 0.9 \\ 2.1 \\
\end{bmatrix}$$
$$A\begin{bmatrix} k \\ m \end{bmatrix} = \bf{b}$$

In [ ]:
import numpy as np

# sampled points
x = np.array([0, 1, 2, 3])
y = np.array([-1, 0.2, 0.9, 2.1])

A = np.vstack([x, np.ones(x.shape)]).T
k, m = np.linalg.lstsq(A, y, rcond=None)[0]

In [ ]:
def plot(x0, y0, x1, y1):
    plt.plot(x0, y0, 'o', label='Original data')
    plt.plot(x1, y1, label='Fitted model')
    plt.legend()
    plt.show()

plot(x, y, x, k*x + m)

### Lattice model
Instead of using a line, lets use the _values_ at specific lattice points as the model. We use a finite grid with a high and low limit. The downside is we can only evaluate it within the bounds, whereas the line model could be evaluated everywhere.

$$
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
x_0 \\ x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
-1 \\ 0.2 \\ 0.9 \\ 2.1 \\
\end{bmatrix}
$$

$$A\bf{x} = \bf{b}$$



In [ ]:
plot(x, y, x, y)

But what if we have more points? And that are not exactly _on_ the lattice grid?

$$f(0) = -1$$
$$f(1) = 0.2$$
$$f(2) = 0.9$$
$$f(2.2) = 1$$
$$f(3) = 2.1$$

Then we do linear interpolation ("lerp"). The closest lattice points are 2 and 3.

$$0.8 \cdot f(2) + 0.2 \cdot f(3) = 1$$

In [ ]:
EPSILON = 1e-8

def value_constraints(xx, n):
    for f, i in zip(*np.modf(x)):
        row = np.zeros(n)
        if 1 - f > EPSILON:
            row[int(i)] = 1 - f
        if f > EPSILON:
            row[int(i) + 1] = f
        yield row
    
x = np.array([0, 1, 2, 2.2, 3])
y = np.array([-1, 0.2, 0.9, 1, 2.1])

A = np.array(list(value_constraints(x, 4)))
A

In [ ]:
xg = np.linalg.lstsq(A, y, rcond=None)[0]
plot(x, y, np.arange(len(xg)), xg)

## Smoothess constraints
We can also coerce the grid model to be more smooth by $f''(x) = 0$. If we aproximate $f''$ by the finite difference $f''(x) = f(⌊x⌋ + 1) - f(⌊x⌋)$ (and likewise for $f''$) we can formulate it like so

$$f(n + 1) - f(n) = f(n + 2) - f(n)$$

For our 4-element grid we get
$$f(1) - f(0) = f(2) - f(1)$$
$$f(2) - f(1) = f(3) - f(2)$$

Or in matrix form

$$
\begin{bmatrix}
1 & -2 & 1 & 0 \\
0 & 1 & -2 & 1 \\
\end{bmatrix}
\begin{bmatrix}
x_0 \\ x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
0 \\ 0 \\
\end{bmatrix}
$$

In [ ]:
def smoothness_constraints(n):
    for i in range(n - 2):
        row = np.zeros(n)
        row[i + 0] = 1
        row[i + 1] = -2
        row[i + 2] = 1
        yield row

A = np.array(list(value_constraints(x, 4)) + list(smoothness_constraints(4)))
A

In [ ]:
xg = np.linalg.lstsq(A, np.hstack([y, np.zeros(4 - 2)]), rcond=None)[0]
plot(x, y, np.arange(len(xg)), xg)

## Gradient constraints
We can also add gradient constraints if these are known. Intead of linear interpolation, just nearest neighgbor can be used. Let's say we know $f'(1.3) = 2$, we can add it as just $f(2) - f(1) = 2$


Or in matrix form

$$
\begin{bmatrix}
0 & -1 & 1 & 0 \\
\end{bmatrix}
\begin{bmatrix}
x_0 \\ x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
2 \\
\end{bmatrix}
$$

## Weights
We can easily weight the different constraints by just multiplying each row by a weight.

## From another dimension
This is easy to extend with more dimenensions. For example in 2D $\nabla f(2.1, 5.8) = \begin{bmatrix} -1 & 3 \end{bmatrix}^T$ just becomes

$$f(3, 6) - f(2, 6) = -1$$
$$f(2, 6) - f(2, 5) =  3$$

In [ ]:
from mini_svg.load import load_svg
shape = load_svg('volumental.svg')

points = np.array([shape.point(t) for t in np.random.random(512)])

plt.plot(points.real, points.imag, 'x', label='volumental.svg')
ax.axis('equal')
plt.show()